HUGGING FACE SENTIMENT ANALYSIS WITH cardiffnlp Roberta MODEL


In [14]:
import nltk  # ? necessary
!pip install sentencepiece  # ? necessary
!pip install emoji
import emoji
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.9 MB/s eta 0:00:00


In [3]:
# for communicating with associated Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# to create "cleaned" dataset for sentiment analysis

import pandas as pd
import re

def clean_tweet(tweet):
    if isinstance(tweet, str):

        # Remove hashtags
        tweet = re.sub(r'#\w+', '', tweet)
        
        # Remove URLs
        tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)

        # Remove HTML tags
        clean = re.compile('<.*?>')
        tweet = re.sub(clean, '', tweet)

        # Remove HTML ref:
        clean = re.compile(r"&[a-z]+;")
        tweet = re.sub(clean,'',tweet)
        
        # Remove emoticons
        emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', tweet)
        tweet = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', '', tweet)

        # Remove emojis
        tweet = emoji.replace_emoji(tweet,'')
        
        # Remove additional whitespace
        tweet = re.sub(r'\s+', ' ', tweet).strip()
        
        return tweet
    else:
        return np.nan

# Load the dataset
df = pd.read_csv('/content/drive/My Drive/NorthAmerica_preprocess_full_1.csv')

df = df.head(500)  # DEBUG 

# Apply the cleaning function to the tweet column
df['cleaned_tweet'] = df['text'].apply(clean_tweet)

# Save the cleaned dataset
df.to_csv('/content/drive/My Drive/cleaned_dataset_1.csv', index=False)


In [ ]:
df.info()

In [8]:
roberta_sentiment_labels = ['Negative', 'Neutral', 'Positive']

In [19]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import string
import re
import pandas as pd

# model has following outputs: Labels: 0 -> Negative; 
  # 1 -> Neutral; 2 -> Positive
SENTIMENT_MODEL_PATH = "cardiffnlp/twitter-roberta-base-sentiment-latest"

sentiment_tokenizer = AutoTokenizer.from_pretrained(SENTIMENT_MODEL_PATH)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(SENTIMENT_MODEL_PATH)

# Load the dataset
df = pd.read_csv('/content/drive/My Drive/cleaned_dataset_1.csv').head(400) # DEBUG

# Iterate over the dataset rows
ct = 0
for index, row in df.iterrows():
    text = row['cleaned_tweet']

    if ct % 50 == 0:
        print(f"on index {ct}")

    # Check if the text is a string
    if isinstance(text, str):

        # Check if the text is empty
        if not text:
            final_sentiment = "unknown"
        else:
            # Tokenize the text
            sentiment_tokenized_text = sentiment_tokenizer(text, padding=True, 
                        truncation=True, max_length=256, return_tensors="pt")

            # Perform sentiment analysis
            sentiment_output = sentiment_model(**sentiment_tokenized_text)
            sentiment_label = sentiment_output.logits.argmax().item()

            try:
                final_sentiment = roberta_sentiment_labels[sentiment_label]
            except IndexError:
                print("problem with sentiment index")
    else:
        final_sentiment = "unknown"

    # Add the final sentiment to the 'sentiment' column
    df.at[index, 'sentiment'] = final_sentiment

    ct += 1

# Save the modified dataset with sentiment column
df.to_csv('/content/drive/My Drive/modified_dataset_1.csv', index=False)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


on index 0
on index 50
on index 100
on index 150
on index 200
on index 250
on index 300
on index 350


In [ ]:
df.info()

In [27]:
import random
num_sam = 10
tst_lst = random.sample(range(df.shape[0]),num_sam)
df_tst = df.iloc[tst_lst]
for ii in range(num_sam):
  print(f"tweet/sentiment: {df_tst['cleaned_tweet'].iloc[ii]} / {df_tst['sentiment'].iloc[ii]}")

tweet/sentiment: twtr users: spend 3hrs/day on product providing immeasurable new biz, connections, capital, friendships fun Also twtr users: $8/mo for extra features: you know, seems excessive- but I'll cont to pay NFLX $15/mo forever never use it psychological rooting of free is STRONG / Positive
tweet/sentiment: Greetings on . May Lord Buddha’s continue to inspire d entire world to strive for peace,harmony,unity n love / Positive
tweet/sentiment: Kathy Hochul won’t remove Alvin Bragg as Manhattan DA. I’ll fire him as my very first Day ONE action. like your wallet, safety, and freedoms depend upon it. / Negative
tweet/sentiment: XMAS POST With strikes disrupting postal services this month, Royal Mail have brought their last posting dates forward. Make sure gifts cards have been sent by the dates below to ensure they arrive on time! More info on Xmas postal services at / Neutral
tweet/sentiment: The COVID vaccine should not be a requirement for students to attend school. The voice of 